In [1]:
import numpy as np
from moviepy.editor import VideoFileClip

In [2]:
anomaly_data = {
    '3': {
        'Blur': [(19, 59), (79, 138)],
        'Motion': [(0, 19), (40, 60), (80, 106), (121, 141)]
    },
    '4': {
        'Overlay': [(17, 28), (55, 64), (174, 185)],
        'Overexposure': [(28, 30), (64, 70)],
        'Blur': [(83, 153)]
    },
    '5': {
        'Overlay': [(662, 675), (679, 690)],
        'Motion': [(3, 4), (324, 325), (454, 455), (507, 508), (508, 560), (547, 662), (714, 718)],
        'Blur': [(161, 221)]
    }
  }

In [3]:
def lighting_change(frame_last, frame_now):
    hist1, _ = np.histogram(frame_last, bins=256, range=(0, 256))
    hist2, _ = np.histogram(frame_now, bins=256, range=(0, 256))

    hist_diff = np.sum(np.abs(hist1 - hist2))
    
    return hist_diff


def blur(frame):
    gradient = np.gradient(frame)
    gradient_variance = np.var(gradient)

    return gradient_variance


def camera_movement(num_changed_pixels, frame_now):

    total_pixels = np.prod(frame_now.shape[:-1])
    percent_changed = (num_changed_pixels / total_pixels) * 100

    return percent_changed

def calculate_file_info(video_files):
    file_info = {}
    for file in video_files:
        clip = VideoFileClip(file)
        file_size = clip.reader.size
        duration = clip.duration
        file_info[file] = {'file_size': file_size, 'duration': duration}
    return file_info

In [11]:
def anomaly_type_detector(anomals, frame_num, fps):
    for anomaly_type, anomaly_periods in anomals.items():
            for period in anomaly_periods:
                start_time, end_time = period
                time_now = (frame_num + 1) / fps # считаем время по кадрам
                if start_time <= time_now <= end_time:
                    return anomaly_type
                elif time_now < start_time:
                    break
    return 'Normal'


def process_frames_with_anomaly(frames, FPS, file_name, file_size, duration):
    features = {'Type': [],
                'Frame': [],
                'Lighting_Changes': [],
                'Blurriness': [],
                'Blocking_Camera': [],
                'Camera_Movement': [],
                'File_Size': [],
                'Duration':[]}
    prev_frame = None

    current_type = 'Normal'
    anomals = anomaly_data[file_name[file_name.rfind('\\')+1:file_name.rfind('.')]]
    count_frames = FPS * duration

    for frame_num, frame in enumerate(frames):
        frame_now = np.dot(frame[..., :3], [0.2989, 0.5870, 0.1140])
        current_type = anomaly_type_detector(anomals, frame_num, FPS)
        if prev_frame is not None:
            frame_delta = prev_frame - frame_now
            threshold = 50
            num_changed_pixels = np.sum(frame_delta > threshold)

            lighting_changes_am = lighting_change(prev_frame, frame_now)
            blurriness = blur(frame_now)
            blocking_camera = num_changed_pixels
            camera_move = camera_movement(num_changed_pixels, frame)
            little_map={
                'Type': current_type,
                'Frame': frame_num,
                'Lighting_Changes': lighting_changes_am,
                'Blurriness': blurriness,
                'Blocking_Camera': blocking_camera,
                'Camera_Movement': camera_move,
                'File_Size': file_size,
                'Duration': duration
            }
            for name, arr in features.items():
                arr.append(little_map[name])
                
        prev_frame = frame_now
        print(file_name, ': ', frame_num, '/', count_frames)
    return features


In [5]:
import glob
train_path="Train/anomaly/*.mp4"
files=glob.glob(train_path)
files

['Train/anomaly\\0.mp4',
 'Train/anomaly\\2.mp4',
 'Train/anomaly\\3.mp4',
 'Train/anomaly\\4.mp4',
 'Train/anomaly\\5.mp4']

In [6]:
files=[
       'Train/anomaly\\4.mp4']

In [12]:
file_info = calculate_file_info(files)
print("file_info",file_info)

def myiter(video_file):
    clip = VideoFileClip(video_file)
    frames = clip.iter_frames()
    FPS = clip.fps
    features = process_frames_with_anomaly(frames, FPS, video_file, file_info[video_file]['file_size'], file_info[video_file]['duration'])
    return features

extracted_features = [myiter(pth) for pth in files]



file_info {'Train/anomaly\\4.mp4': {'file_size': [800, 600], 'duration': 202.52}}
Train/anomaly\4.mp4 :  0 / 5063.0
Train/anomaly\4.mp4 :  1 / 5063.0
Train/anomaly\4.mp4 :  2 / 5063.0
Train/anomaly\4.mp4 :  3 / 5063.0
Train/anomaly\4.mp4 :  4 / 5063.0
Train/anomaly\4.mp4 :  5 / 5063.0
Train/anomaly\4.mp4 :  6 / 5063.0
Train/anomaly\4.mp4 :  7 / 5063.0
Train/anomaly\4.mp4 :  8 / 5063.0
Train/anomaly\4.mp4 :  9 / 5063.0
Train/anomaly\4.mp4 :  10 / 5063.0
Train/anomaly\4.mp4 :  11 / 5063.0
Train/anomaly\4.mp4 :  12 / 5063.0
Train/anomaly\4.mp4 :  13 / 5063.0
Train/anomaly\4.mp4 :  14 / 5063.0
Train/anomaly\4.mp4 :  15 / 5063.0
Train/anomaly\4.mp4 :  16 / 5063.0
Train/anomaly\4.mp4 :  17 / 5063.0
Train/anomaly\4.mp4 :  18 / 5063.0
Train/anomaly\4.mp4 :  19 / 5063.0
Train/anomaly\4.mp4 :  20 / 5063.0
Train/anomaly\4.mp4 :  21 / 5063.0
Train/anomaly\4.mp4 :  22 / 5063.0
Train/anomaly\4.mp4 :  23 / 5063.0
Train/anomaly\4.mp4 :  24 / 5063.0
Train/anomaly\4.mp4 :  25 / 5063.0
Train/anomaly\4.mp

In [13]:
extracted_features

[{'Type': ['Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   'Normal',
   '

In [14]:
import pandas as pd
df=pd.DataFrame(extracted_features[0])

In [15]:
df

,Type,Frame,Lighting_Changes,Blurriness,Blocking_Camera,Camera_Movement,File_Size,Duration
0,Normal,1,8182,94.120460,2,0.000417,"[800, 600]",202.52
1,Normal,2,8428,94.405410,0,0.000000,"[800, 600]",202.52
2,Normal,3,7214,94.583901,2,0.000417,"[800, 600]",202.52
3,Normal,4,6646,94.763936,0,0.000000,"[800, 600]",202.52
4,Normal,5,5954,94.870887,1,0.000208,"[800, 600]",202.52
...,...,...,...,...,...,...,...,...
5057,Normal,5058,6986,94.216735,0,0.000000,"[800, 600]",202.52
5058,Normal,5059,6688,94.397759,0,0.000000,"[800, 600]",202.52
5059,Normal,5060,7126,94.528023,0,0.000000,"[800, 600]",202.52
5060,Normal,5061,5998,94.628859,2,0.000417,"[800, 600]",202.52


In [16]:
df.describe()

,Frame,Lighting_Changes,Blurriness,Blocking_Camera,Camera_Movement,Duration
count,5062.000000,5062.000000,5062.000000,5062.000000,5062.000000,5062.00
mean,2531.500000,14940.177400,58.460087,739.577242,0.154079,202.52
std,1461.417862,24646.870334,37.150470,5731.870994,1.194140,0.00
min,1.000000,514.000000,0.204470,0.000000,0.000000,202.52
25%,1266.250000,7142.000000,16.731738,0.000000,0.000000,202.52
50%,2531.500000,9205.000000,67.041611,0.000000,0.000000,202.52
75%,3796.750000,13448.500000,95.064935,2.000000,0.000417,202.52
max,5062.000000,579370.000000,99.471869,273496.000000,56.978333,202.52


In [17]:
dfs=[pd.DataFrame(v) for v in extracted_features]

In [18]:
for name_mp4,df in zip(files,dfs):
    name=name_mp4.split('\\')[-1]
    df.to_csv(f"csv/anomaly/{name}.csv")